In [22]:
import cv2
import numpy as np
import pickle
import pandas as pd
from ultralytics import YOLO
import cvzone


my_file = open("coco.txt", "r")
data = my_file.read()
class_list = data.split("\n")

model=YOLO('yolov8s.pt')

with open("points_and_names_of_area",'rb') as file:
                data = pickle.load(file)
                polylines , areaName = data['polylines'] , data['areaName']
            
cap=cv2.VideoCapture("ParkWatch.mp4")


count=0

while True:
    ret, frame = cap.read()
    if not ret:
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        continue
   
    count += 1
    if count % 3 != 0:
        continue

    frame=cv2.resize(frame,(1200,700))
    frame_copy = frame.copy()
    results=model.predict(frame)
    a=results[0].boxes.data.cpu().numpy()
    px=pd.DataFrame(a).astype("float")
    
    car_coordinates=[]
    for index,car_data in px.iterrows():
 
        x1, y1, x2, y2, class_index = map(int, [car_data[0], car_data[1], car_data[2], car_data[3], car_data[5]])
        class_name =class_list[class_index]

        center_x=int(x1+x2)//2
        center_y=int(y1+y2)//2

        if 'car' in class_name:
            car_coordinates.append([center_x,center_y])
            #cv2.rectangle(frame,(x1,y1),(x2,y2),(255,0,0),2)
            
    counter1 = []        
    for i, polyline in enumerate (polylines):
        cv2.polylines(frame, [polyline], True, (0, 255, 0), 2)
        cvzone.putTextRect(frame,f'{areaName[i]}' , tuple(polyline[0]),1,1)
        
        for center in car_coordinates:
            center_x1 = center[0]
            center_y1 = center[1]
            # >0 means =1 means car in polyline
            if cv2.pointPolygonTest(polyline,(center_x1,center_y1),False)  >=0: # check if car enter region of interest 
                cv2.circle(frame,(center_x1,center_y1),2,(0,255,0),-1) # draw green circle in center of car
                cv2.polylines(frame, [polyline], True, (0, 0, 255), 2) # When the car enters the polyline change color from green to red.
                counter1.append(center_x1)
    num_of_AllParkimg = 7 
    car_count = len(counter1)
    # colorT-> color of texT, colorR-> color of Rectangle
    cvzone.putTextRect(frame,f'busy spots is {car_count} ' , (50,40), 2, 2, colorT=(0, 0, 0), colorR=(0, 0, 255))
    free_space = num_of_AllParkimg - car_count
    cvzone.putTextRect(frame,f'free spots is {free_space} ' ,(50,90),2,2, colorT=(0, 0, 0), colorR=(0, 255, 0))
    
    cv2.imshow('FRAME', frame)
    key = cv2.waitKey(60)
    if key == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()


0: 384x640 9 cars, 48.1ms
Speed: 6.7ms preprocess, 48.1ms inference, 6.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 bus, 2 trucks, 12.4ms
Speed: 6.7ms preprocess, 12.4ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 16.3ms
Speed: 0.0ms preprocess, 16.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 16.6ms
Speed: 0.0ms preprocess, 16.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 6.5ms
Speed: 2.7ms preprocess, 6.5ms inference, 8.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 bus, 21.6ms
Speed: 0.0ms preprocess, 21.6ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 bus, 9.6ms
Speed: 0.0ms preprocess, 9.6ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 16.6ms
Speed: 0.0ms preprocess, 16.6ms inference, 3.0ms postprocess per 